In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import json
import gdown
import sys
from tqdm import tqdm
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

# pd.set_option('display.max_rows', None)
# pd.set_option('mode.chained_assignment',None)

In [2]:
df1 = pd.read_csv('Domain_click_1.csv')
df2 = pd.read_csv('Domain_click_2.csv')
df3 = pd.read_csv('Domain_click_3.csv')
# df4 = pd.read_csv('Domain_click_4.csv')
# df5 = pd.read_csv('/content/Domain extract/Domain_click_5.csv')
# df6 = pd.read_csv('/content/Domain extract/Domain_click_6.csv')

C:\Users\PM\AppData\Local\Temp\ipykernel_12680\1039586624.py:1: DtypeWarning: Columns (29,32,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Domain_click_1.csv')
C:\Users\PM\AppData\Local\Temp\ipykernel_12680\1039586624.py:2: DtypeWarning: Columns (26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Domain_click_2.csv')
C:\Users\PM\AppData\Local\Temp\ipykernel_12680\1039586624.py:3: DtypeWarning: Columns (23,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('Domain_click_3.csv')


In [3]:
suburbdf = pd.DataFrame()
suburbdf = pd.concat([suburbdf,df1],ignore_index =True)
suburbdf = pd.concat([suburbdf,df2],ignore_index =True)
suburbdf = pd.concat([suburbdf,df3],ignore_index =True)
# suburbdf = pd.concat([suburbdf,df4],ignore_index =True)
# suburbdf = pd.concat([suburbdf,df5],ignore_index =True)
suburbdf

,Bedrooms,Type,Median Price,Avg Days on Market,Clearance Rate,Sold This Year,POPULATION,AVERAGE AGE,OWNER,RENTER,...,no_of_sales_2022,median_price_2021,growth_rate_2021,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019
0,3.0,House,$625k,87 days,-,12.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.0,$405k,1.3%,18.0,$400k,11.1%,15.0,NaN,NaN,NaN
2,4.0,House,$685k,65 days,-,47.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,118.0,$458k,4.8%,90.0,$437k,-7.2%,64.0,NaN,NaN,NaN
4,5.0,House,-,-,-,8.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.0,$565k,0.5%,31,$562k,-6.3%,50.0,NaN,NaN,NaN
51272,4.0,House,$975k,57 days,-,54.0,"8,226",40 to 59,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72.0,$715k,4.5%,76,$684k,-2.3%,64.0,NaN,NaN,NaN
51274,5.0,House,$1.21m,65 days,-,11.0,"8,226",40 to 59,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = suburbdf

In [5]:
len(suburbdf['SUBURB'].unique())

6444

# checkpoint

In [6]:
df.columns

Index(['Bedrooms', 'Type', 'Median Price', 'Avg Days on Market',
       'Clearance Rate', 'Sold This Year', 'POPULATION', 'AVERAGE AGE',
       'OWNER', 'RENTER', 'FAMILY', 'SINGLE', 'SUBURB', 'Time_stamp',
       'Sold this year: ', 'Rental median price: ',
       'Auction clearance (Higher = more competition): ',
       'Average days on market (Lower = more competition): ',
       'median_price_2024', 'growth_rate_2024', 'no_of_sales_2024',
       'median_price_2023', 'growth_rate_2023', 'no_of_sales_2023',
       'median_price_2022', 'growth_rate_2022', 'no_of_sales_2022',
       'median_price_2021', 'growth_rate_2021', 'no_of_sales_2021',
       'median_price_2020', 'growth_rate_2020', 'no_of_sales_2020',
       'median_price_2019', 'growth_rate_2019', 'no_of_sales_2019'],
      dtype='object')

In [7]:
selected = df.iloc[:,14:].columns
df[selected] = df[selected].shift(-1)
df = df.dropna(axis=0, how='all')
df

,Bedrooms,Type,Median Price,Avg Days on Market,Clearance Rate,Sold This Year,POPULATION,AVERAGE AGE,OWNER,RENTER,...,no_of_sales_2022,median_price_2021,growth_rate_2021,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019
0,3.0,House,$625k,87 days,-,12.0,"3,103",20 to 39,63%,37%,...,22.0,$405k,1.3%,18.0,$400k,11.1%,15.0,NaN,NaN,NaN
2,4.0,House,$685k,65 days,-,47.0,"3,103",20 to 39,63%,37%,...,118.0,$458k,4.8%,90.0,$437k,-7.2%,64.0,NaN,NaN,NaN
4,5.0,House,-,-,-,8.0,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.0,House,-,-,-,1.0,25,40 to 59,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,House,$360k,76 days,-,14.0,"9,701",60+,68%,32%,...,30.0,$237k,20.9%,21.0,$196k,-4.4%,26.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51266,5.0,House,$1.21m,65 days,-,11.0,"8,226",40 to 59,85%,15%,...,16.0,$795k,12%,11,$710k,-,13.0,NaN,NaN,NaN
51268,2.0,House,-,-,-,1.0,"8,226",40 to 59,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51270,3.0,House,$800k,48 days,-,38.0,"8,226",40 to 59,85%,15%,...,41.0,$565k,0.5%,31,$562k,-6.3%,50.0,NaN,NaN,NaN
51272,4.0,House,$975k,57 days,-,54.0,"8,226",40 to 59,85%,15%,...,72.0,$715k,4.5%,76,$684k,-2.3%,64.0,NaN,NaN,NaN


In [8]:
df['Postcode'] = df['SUBURB'].apply(lambda x: x[-4:].title())
df['State'] = df['SUBURB'].apply(lambda x: x.split('-')[-2].title())
df['Suburb Name'] = df['SUBURB'].apply(lambda x: ' '.join(x.split('-')[:-2]).title())
df['Postcode'] = df['Postcode'].apply(lambda x: str(x).zfill(4))

C:\Users\PM\AppData\Local\Temp\ipykernel_12680\1786961768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Postcode'] = df['SUBURB'].apply(lambda x: x[-4:].title())
C:\Users\PM\AppData\Local\Temp\ipykernel_12680\1786961768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = df['SUBURB'].apply(lambda x: x.split('-')[-2].title())
C:\Users\PM\AppData\Local\Temp\ipykernel_12680\1786961768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [9]:
state_names = {
    'Tas': 'Tasmania',
    'Nsw': 'New South Wales',
    'Qld': 'Queensland',
    'Nt': 'Northern Territory',
    'Vic': 'Victoria',
    'Sa': 'South Australia',
    'Act': 'Australian Capital Territory',
    'Wa': 'Western Australia'
    }
df['State'] = df['State'].replace(state_names)

C:\Users\PM\AppData\Local\Temp\ipykernel_12680\2540312763.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = df['State'].replace(state_names)


In [10]:
new_column_names = {col: col.lower().replace(' ', '_') for col in df.columns}
df = df.rename(columns=new_column_names)

In [11]:
df['avg_days_on_market'] = df['avg_days_on_market'].apply(lambda x: np.nan if x == '-' else str(x).replace(' days', ''))

In [12]:
# df[['median_price','median_price_2018','median_price_2019','median_price_2020','median_price_2021','median_price_2022','median_price_2023']]

In [13]:
# df[['median_price','median_price_2018','median_price_2019','median_price_2020','median_price_2021','median_price_2022','median_price_2023']]

In [14]:
del df[df.columns[14]]
df = df.rename(columns = {df.columns[14]:'rental_median_price',df.columns[15]:'auction_clearance',df.columns[16]:'average_days_on_market'})

In [15]:
def convert_to_float(value):
    try:

      multiplier = 1
      if value.endswith('k'):
          multiplier = 1000
      elif value.endswith('m'):
          multiplier = 1000000
      value = re.sub('[^\d.]', '', value)
      return (float(value) * multiplier)
    except:
      None

# columns_to_clean = ['median_price','rental_median_price', 'median_price_2018', 'median_price_2019', 'median_price_2020', 'median_price_2021', 'median_price_2022', 'median_price_2023']
columns_to_clean = ['median_price','rental_median_price', 'median_price_2019', 'median_price_2020', 'median_price_2021', 'median_price_2022', 'median_price_2023']
for column in columns_to_clean:
    df[column] = df[column].astype(str).apply(lambda x: x.replace('$', '') if pd.notnull(x) else x)
    df[column] = [convert_to_float(value) for value in df[column]]

# df[['median_price','rental_median_price','median_price_2018','median_price_2019','median_price_2020','median_price_2021','median_price_2022','median_price_2023']]

In [16]:
df[['growth_rate_2019','growth_rate_2020','growth_rate_2021','growth_rate_2022','growth_rate_2023']]

,growth_rate_2019,growth_rate_2020,growth_rate_2021,growth_rate_2022,growth_rate_2023
0,NaN,11.1%,1.3%,22.2%,13.1%
2,NaN,-7.2%,4.8%,27.3%,7.2%
4,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
8,NaN,-4.4%,20.9%,22.4%,17.2%
...,...,...,...,...,...
51266,NaN,-,12%,21.8%,15.7%
51268,NaN,NaN,NaN,NaN,NaN
51270,NaN,-6.3%,0.5%,23.5%,13.9%
51272,NaN,-2.3%,4.5%,28.7%,8.2%


In [17]:
# columns_to_clean = ['growth_rate_2018','growth_rate_2019','growth_rate_2020','growth_rate_2021','growth_rate_2022','growth_rate_2023']
columns_to_clean = ['growth_rate_2019','growth_rate_2020','growth_rate_2021','growth_rate_2022','growth_rate_2023']
for column in columns_to_clean:
    df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
# df[['growth_rate_2018','growth_rate_2019','growth_rate_2020','growth_rate_2021','growth_rate_2022','growth_rate_2023']]

In [18]:
df['avg_days_on_market'] = df['avg_days_on_market'].astype(float)
df['clearance_rate'] = df['clearance_rate'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['population'] = df['population'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace(',', '').replace('+', '')))
df['owner'] = df['owner'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['renter'] = df['renter'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['family'] = df['family'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['single'] = df['single'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))
df['auction_clearance'] = df['auction_clearance'].apply(lambda x: np.NaN if x == '-' else float(str(x).replace('%','')))

In [19]:
del df['average_days_on_market']

In [20]:
# columns_to_clean = ['no_of_sales_2018','no_of_sales_2019','no_of_sales_2020','no_of_sales_2021','no_of_sales_2022','no_of_sales_2023']
columns_to_clean = ['no_of_sales_2019','no_of_sales_2020','no_of_sales_2021','no_of_sales_2022','no_of_sales_2023']
for column in columns_to_clean:
    df[column] = df[column].apply(lambda x: np.NaN if x == '-' else float(x))


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25638 entries, 0 to 51274
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   bedrooms             25638 non-null  float64
 1   type                 25638 non-null  object 
 2   median_price         9254 non-null   float64
 3   avg_days_on_market   7977 non-null   float64
 4   clearance_rate       2563 non-null   float64
 5   sold_this_year       25638 non-null  float64
 6   population           24792 non-null  float64
 7   average_age          24768 non-null  object 
 8   owner                24757 non-null  float64
 9   renter               24757 non-null  float64
 10  family               24495 non-null  float64
 11  single               24495 non-null  float64
 12  suburb               25638 non-null  object 
 13  time_stamp           25638 non-null  object 
 14  rental_median_price  7620 non-null   float64
 15  auction_clearance    2549 non-null  

In [22]:
df['postcode'] = df['postcode'].apply(lambda x: str(x).zfill(4))

In [23]:
df

,bedrooms,type,median_price,avg_days_on_market,clearance_rate,sold_this_year,population,average_age,owner,renter,...,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019,postcode,state,suburb_name
0,3.0,House,625000.0,87.0,NaN,12.0,3103.0,20 to 39,63.0,37.0,...,18.0,400000.0,11.1,15.0,None,NaN,NaN,4879,Queensland,Trinity Park
2,4.0,House,685000.0,65.0,NaN,47.0,3103.0,20 to 39,63.0,37.0,...,90.0,437000.0,-7.2,64.0,None,NaN,NaN,4879,Queensland,Trinity Park
4,5.0,House,NaN,NaN,NaN,8.0,3103.0,20 to 39,63.0,37.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,4879,Queensland,Trinity Park
6,2.0,House,NaN,NaN,NaN,1.0,25.0,40 to 59,63.0,37.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,4570,Queensland,Melawondi
8,2.0,House,360000.0,76.0,NaN,14.0,9701.0,60+,68.0,32.0,...,21.0,196000.0,-4.4,26.0,None,NaN,NaN,3305,Victoria,Portland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51266,5.0,House,1210000.0,65.0,NaN,11.0,8226.0,40 to 59,85.0,15.0,...,11.0,710000.0,NaN,13.0,None,NaN,NaN,2530,New South Wales,Horsley
51268,2.0,House,NaN,NaN,NaN,1.0,8226.0,40 to 59,85.0,15.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,2530,New South Wales,Horsley
51270,3.0,House,800000.0,48.0,NaN,38.0,8226.0,40 to 59,85.0,15.0,...,31.0,562000.0,-6.3,50.0,None,NaN,NaN,2530,New South Wales,Horsley
51272,4.0,House,975000.0,57.0,NaN,54.0,8226.0,40 to 59,85.0,15.0,...,76.0,684000.0,-2.3,64.0,None,NaN,NaN,2530,New South Wales,Horsley


In [24]:
df.columns

Index(['bedrooms', 'type', 'median_price', 'avg_days_on_market',
       'clearance_rate', 'sold_this_year', 'population', 'average_age',
       'owner', 'renter', 'family', 'single', 'suburb', 'time_stamp',
       'rental_median_price', 'auction_clearance', 'median_price_2024',
       'growth_rate_2024', 'no_of_sales_2024', 'median_price_2023',
       'growth_rate_2023', 'no_of_sales_2023', 'median_price_2022',
       'growth_rate_2022', 'no_of_sales_2022', 'median_price_2021',
       'growth_rate_2021', 'no_of_sales_2021', 'median_price_2020',
       'growth_rate_2020', 'no_of_sales_2020', 'median_price_2019',
       'growth_rate_2019', 'no_of_sales_2019', 'postcode', 'state',
       'suburb_name'],
      dtype='object')

In [25]:
df = df.reset_index(drop=True)

In [26]:
df = df.sort_values(by=['postcode', 'suburb_name'])

In [27]:
df['rental_yield'] = round((df['rental_median_price'] * 12 / df['median_price']) * 100,2)

In [28]:
custom = ['postcode', 'state','suburb_name','bedrooms', 'type', 'rental_yield','median_price', 'avg_days_on_market',
       'clearance_rate', 'sold_this_year', 'population', 'average_age',
       'owner', 'renter', 'family', 'single', 'suburb', 'time_stamp',
       'rental_median_price', 'auction_clearance', 'median_price_2023',
       'growth_rate_2023', 'no_of_sales_2023', 'median_price_2022',
       'growth_rate_2022', 'no_of_sales_2022', 'median_price_2021',
       'growth_rate_2021', 'no_of_sales_2021', 'median_price_2020',
       'growth_rate_2020', 'no_of_sales_2020', 'median_price_2019',
       'growth_rate_2019', 'no_of_sales_2019', 'median_price_2018',
       'growth_rate_2018', 'no_of_sales_2018']
df = df.reindex(columns = custom)
df

,postcode,state,suburb_name,bedrooms,type,rental_yield,median_price,avg_days_on_market,clearance_rate,sold_this_year,...,no_of_sales_2021,median_price_2020,growth_rate_2020,no_of_sales_2020,median_price_2019,growth_rate_2019,no_of_sales_2019,median_price_2018,growth_rate_2018,no_of_sales_2018
57,0800,Northern Territory,Darwin City,2.0,House,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
58,0800,Northern Territory,Darwin City,4.0,House,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
59,0800,Northern Territory,Darwin City,1.0,Unit,2.24,260000.0,143.0,NaN,67.0,...,25.0,210000.0,-8.3,24.0,None,NaN,NaN,NaN,NaN,NaN
60,0800,Northern Territory,Darwin City,2.0,Unit,1.75,412000.0,169.0,NaN,89.0,...,61.0,310000.0,-12.7,51.0,None,NaN,NaN,NaN,NaN,NaN
61,0800,Northern Territory,Darwin City,3.0,Unit,1.67,540000.0,115.0,NaN,55.0,...,45.0,450000.0,1.1,39.0,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10643,7470,Tasmania,Rosebery,2.0,House,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
10644,7470,Tasmania,Rosebery,3.0,House,NaN,187000.0,69.0,NaN,25.0,...,11.0,NaN,NaN,8.0,None,NaN,NaN,NaN,NaN,NaN
10645,7470,Tasmania,Rosebery,4.0,House,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
10646,7470,Tasmania,Rosebery,2.0,Unit,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


In [29]:
australia_timezone = pytz.timezone('Australia/Sydney')
current_time = datetime.datetime.now(australia_timezone)
filename = 'Domain_Extract_'+ str(current_time.date()).replace('-', '')+'.xlsx'
df.to_excel(filename,index = False)

In [30]:
df = pd.read_excel(filename)

In [31]:
# df= pd.read_excel('Domain_Extract_20230924.xlsx')

In [32]:
df['postcode'] = df['postcode'].apply(lambda x: str(x).zfill(4))

In [33]:
l = '''
postcode  char(4) 
state text 
suburb_name text 
bedrooms int 
type text 
median_price text 
median_price_int int 
rental_median_price int 
rental_yield double 
avg_days_on_market int 
clearance_rate double 
sold_this_year int 
population int 
average_age text 
owner double 
renter double 
family double 
single double 
domain_search_string text 
data_stamp text 
auction_clearance double 
average_days_on_market double 
median_price_2023 int 
growth_rate_2023 double 
no_of_sales_2023 double 
median_price_2022 int 
growth_rate_2022 double 
no_of_sales_2022 double 
median_price_2021 int 
growth_rate_2021 double 
no_of_sales_2021 double 
median_price_2020 int 
growth_rate_2020 double 
no_of_sales_2020 double 
median_price_2019 int 
growth_rate_2019 double 
no_of_sales_2019 double 
suburb_id int 
max_bedrooms int 
state_code varchar(3) 
longitude double 
latitude double
'''

In [34]:
l = l.split('\n')

In [35]:
# for i in range(len(l)):
#     print(l[i].strip().split(' '))

In [36]:
subdf = pd.read_excel('suburbs.xlsx')
subdf = subdf[['Unnamed: 0','state','domain_search_string']]
subdf.rename(columns = {'Unnamed: 0':'suburb_id','state':'state_code'},inplace=True)

In [37]:
mapcols = {'Northern Territory':'NT','New South Wales':'NSW','Australian Capital Territory':'ACT',
 'Victoria':'VIC','Queensland':'QLD','South Australia':'SA','Western Australia':'WA','Tasmania':'TAS'}
subdf['state_code'] = subdf['state_code'].replace(mapcols)

In [38]:
df.rename(columns={'median_price':'median_price_int','suburb':'domain_search_string','time_stamp':'data_stamp'}, inplace= True)
df.drop(columns=['median_price_2018','growth_rate_2018','no_of_sales_2018'], inplace = True)

In [39]:
latdf = pd.read_excel('Latitude_and_Longitude_Extract_20230802.xlsx')

In [40]:
latdf.rename(columns={'lattitude':'latitude'},inplace=True)

In [41]:
df = pd.merge(df,subdf,how = 'left',left_on='domain_search_string',right_on='domain_search_string')

In [42]:
latdf.rename(columns={'Search_string':'domain_search_string'},inplace=True)
latdf.drop(columns=['lat_and_lon'],inplace=True)
df = pd.merge(df,latdf,how = 'left',left_on='domain_search_string',right_on='domain_search_string')

In [43]:
df['median_price_80'] = 0.8 * df['median_price_int']

In [44]:
max_bedrooms_df = df.groupby(['domain_search_string'])['bedrooms'].max().reset_index()
max_bedrooms_df.rename(columns={'bedrooms': 'max_bedrooms'}, inplace=True)
df = pd.merge(df, max_bedrooms_df, on=['domain_search_string'], how='left')

In [45]:
# a = list(df.columns)
# for i in range(len(a)):
#     print(i,'-',a[i],'-------',df[a[i]].dtype)

In [46]:
from sqlalchemy import create_engine
import mysql.connector

# Define your database connection parameters
host = "54.79.186.42"
user = "root"
password = "secondstorey"
database = "secondstorey"

# Establish a connection to the database
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Check if the connection was successful
if connection.is_connected():
    print("Connected to MySQL database")

# Close the connection when done
# Define the SQLAlchemy connection string
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Create the table in MySQL (if_exists='replace' will create the table if it doesn't exist)
df.to_sql(name='domain_insights', con=engine, if_exists='replace', index=False)
engine.dispose()

Connected to MySQL database


In [47]:
'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21197254&cid=0F87FC3AA6EB46E0'

'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21197254&cid=0F87FC3AA6EB46E0'